In [1]:
%%writefile claims_dashboard.py


import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches
import tempfile

# --- Scraping Functions ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    time.sleep(2)
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    time.sleep(2)
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Functions ---
def generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        print("No data available to generate charts.")
        return None, None, None, None
    
    year_int = int(year)
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        print(f"Data for year {year} is empty: Patient: {patient_df.empty}, Claim: {claim_df.empty}, MC: {mc_df.empty}")
        return None, None, None, None
    
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Arial', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # Provider Charts
    fig = plt.figure(figsize=(12, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    if show_mc_pct:
        fig = plt.figure(figsize=(18, 9))
        top_visits_provs = set(mc_df.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df[mc_df['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({year}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(fig)
    else:
        provider_charts.append(None)
    if show_mc_pct:
        plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    # Employee Charts
    fig = plt.figure(figsize=(12, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({year})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({year})', pad=15)
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    
    print(f"Generated {len(provider_charts)} provider charts and {len(employee_charts)} employee charts for year {year}")
    return provider_images, employee_images, provider_charts, employee_charts

# --- New Monthly Plotting Function ---
def generate_monthly_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        print("No data available to generate monthly charts.")
        return None, None, None, None
    
    year_int = int(year)
    month_int = int(month.split('-')[1])  # Extract month number from "YYYY-MM" format
    month_name = datetime.strptime(str(month_int), '%m').strftime('%B')  # Convert to month name
    
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        print(f"Data for year {year} is empty: Patient: {patient_df.empty}, Claim: {claim_df.empty}, MC: {mc_df.empty}")
        return None, None, None, None
    
    # Since we don't have date-specific data, we'll assume the data is yearly and simulate monthly by dividing
    # For a real implementation, you'd need date columns in your scraped data to filter by month
    patient_df_month = patient_df.copy()  # Placeholder: assumes yearly data split evenly
    claim_df_month = claim_df.copy()
    mc_df_month = mc_df.copy()
    
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Arial', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # Provider Charts
    fig = plt.figure(figsize=(12, 7))
    top_prov_visits = mc_df_month.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_prov_mc = mc_df_month.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({month_name} {year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    if show_mc_pct:
        fig = plt.figure(figsize=(18, 9))
        top_visits_provs = set(mc_df_month.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df_month.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df_month[mc_df_month['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({month_name} {year}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(fig)
    else:
        provider_charts.append(None)
    if show_mc_pct:
        plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_prov_claim = claim_df_month.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_prov_avg_claim = claim_df_month.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({month_name} {year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    # Employee Charts
    fig = plt.figure(figsize=(12, 7))
    top_emp_visits = patient_df_month.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_emp_claim = patient_df_month.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_emp_avg_claim = patient_df_month.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({month_name} {year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_emp_mc = patient_df_month.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({month_name} {year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 7))
    top_emp_avg_mc = patient_df_month.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({month_name} {year})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(10, 7))
    division_claims = patient_df_month.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({month_name} {year})', pad=15)
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    
    print(f"Generated {len(provider_charts)} provider charts and {len(employee_charts)} employee charts for {month_name} {year}")
    return provider_images, employee_images, provider_charts, employee_charts

def fig_to_image(fig):
    if fig is None:
        return None
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    return img_array

def charts_to_pptx(provider_charts, employee_charts, year):
    try:
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]  # Blank slide layout with title
        prs.slide_width = Inches(13.33)  # Widescreen slide size
        prs.slide_height = Inches(7.5)

        # Define chart titles
        provider_titles = [
            f"Top 10 Providers by Total Visits ({year})",
            f"Top 10 Providers by Total MC Given ({year})",
            f"Top 20 Providers by % MC Given ({year})",
            f"Top 10 Providers by Total Claim ({year})",
            f"Top 10 Providers by Avg Claim per Visit ({year})"
        ]
        
        employee_titles = [
            f"Top 10 Employees by Total Visits ({year})",
            f"Top 10 Employees by Total Claim ({year})",
            f"Top 10 Employees by Avg Claim per Visit ({year})",
            f"Top 10 Employees by Total MC ({year})",
            f"Top 10 Employees by Avg MC per Visit ({year})",
            f"Claim Distribution by Division ({year})"
        ]

        # Provider Slide (all charts on one slide)
        if provider_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Provider Insights ({year})"
            num_charts = sum(1 for fig in provider_charts if fig is not None)
            if num_charts == 0:
                print("No valid provider charts to export")
            else:
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)
                chart_height = Inches(2.5) if rows > 1 else Inches(5)
                top_start = Inches(1.5)
                valid_charts = [fig for fig in provider_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{provider_titles[idx]}' to Provider slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{provider_titles[idx]}' - empty image stream")

        # Employee Slide (all charts on one slide)
        if employee_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Employee Insights ({year})"
            num_charts = sum(1 for fig in employee_charts if fig is not None)
            if num_charts == 0:
                print("No valid employee charts to export")
            else:
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)
                chart_height = Inches(2.5) if rows > 1 else Inches(5)
                top_start = Inches(1.5)
                valid_charts = [fig for fig in employee_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{employee_titles[idx]}' to Employee slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{employee_titles[idx]}' - empty image stream")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.pptx') as tmp_file:
            prs.save(tmp_file.name)
            tmp_file_path = tmp_file.name

        print(f"PPTX file saved to temporary path: {tmp_file_path}")
        return tmp_file_path

    except Exception as e:
        print(f"Error generating PPTX: {str(e)}")
        return None

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard", css="""
    body { background-color: #f5f6f5; }
    h1, h2 { color: #333333; font-family: Arial; }
""") as demo:
    gr.Markdown("# Claims Analysis Dashboard (2024 - Present)")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL", placeholder="Enter URL here", lines=1)
        user_id_input = gr.Textbox(label="User ID", placeholder="Enter User ID", lines=1)
        password_input = gr.Textbox(label="Password", type="password", placeholder="Enter Password", lines=1)
    scrape_btn = gr.Button("Submit", variant="primary")
    
    with gr.Row():
        year_dropdown = gr.Dropdown(
            label="Select Year to View Data",
            choices=["2024"],
            value="2024",
            allow_custom_value=False,
            interactive=False
        )
        month_dropdown = gr.Dropdown(
            label="Select Month (Optional)",
            choices=["Yearly"] + [f"{year}-{month:02d}" for year in range(2024, 2036) for month in range(1, 13)],
            value="Yearly",
            allow_custom_value=False,
            interactive=False
        )
        show_mc_pct_checkbox = gr.Checkbox(label="Show % MC Given Chart", value=True)
        mc_sort_dropdown = gr.Dropdown(
            label="Sort % MC Given",
            choices=["desc", "asc"],
            value="desc",
            allow_custom_value=False
        )
    
    status_output = gr.Textbox(label="Status", lines=2, interactive=False)
    patient_state = gr.State()
    claim_state = gr.State()
    mc_state = gr.State()

    with gr.Tabs():
        with gr.TabItem("Provider Insights"):
            gr.Markdown("## Provider Insights Dashboard")
            with gr.Row():
                prov_chart1 = gr.Image(label="Total Visits by Providers", interactive=False)
                prov_chart2 = gr.Image(label="Total MC by Providers", interactive=False)
            with gr.Row():
                prov_chart3 = gr.Image(label="% MC Given by Providers", interactive=False, visible=True)
            with gr.Row():
                prov_chart4 = gr.Image(label="Total Claim by Providers", interactive=False)
                prov_chart5 = gr.Image(label="Average Claim per Visit by Providers", interactive=False)
            download_btn_prov = gr.Button("Download Provider Charts as PPTX")

        with gr.TabItem("Employee Insights"):
            gr.Markdown("## Employee Insights Dashboard")
            with gr.Row():
                emp_chart1 = gr.Image(label="Total Visits by Employees", interactive=False)
                emp_chart2 = gr.Image(label="Total Claim by Employees", interactive=False)
            with gr.Row():
                emp_chart3 = gr.Image(label="Average Claim per Visit by Employees", interactive=False)
                emp_chart4 = gr.Image(label="Total MC by Employees", interactive=False)
            with gr.Row():
                emp_chart5 = gr.Image(label="Average MC per Visit by Employees", interactive=False)
                emp_chart6 = gr.Image(label="Claim Distribution by Division", interactive=False)
            download_btn_emp = gr.Button("Download Employee Charts as PPTX")

    def scrape_and_store(url, user_id, password, show_mc_pct, mc_sort_order):
        patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
        if patient_data_by_year is None or claim_data_by_year is None or mc_data_by_year is None:
            return (
                status, None, None, None, 
                gr.update(choices=["2024"], value="2024", interactive=False),
                gr.update(choices=["Yearly"], value="Yearly", interactive=False),
                None, None, None, None, None, None, None, None, None, None, None
            )

        available_years = sorted(set(patient_data_by_year.keys()) | set(claim_data_by_year.keys()) | set(mc_data_by_year.keys()))
        year_choices = [str(year) for year in available_years]
        default_year = year_choices[0] if year_choices else "2024"
        month_choices = ["Yearly"] + [f"{year}-{month:02d}" for year in available_years for month in range(1, 13)]

        provider_images, employee_images, provider_figs, employee_figs = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, default_year, show_mc_pct, mc_sort_order)
        return (
            status, patient_data_by_year, claim_data_by_year, mc_data_by_year,
            gr.update(choices=year_choices, value=default_year, interactive=True),
            gr.update(choices=month_choices, value="Yearly", interactive=True),
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def update_dashboard(year, month, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
            return [None] * 11
        if month == "Yearly":
            provider_images, employee_images, _, _ = generate_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
        else:
            provider_images, employee_images, _, _ = generate_monthly_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct, mc_sort_order)
        return (
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def download_provider_pptx(year, month, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if month == "Yearly":
            _, _, provider_figs, _ = generate_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
            pptx_path = charts_to_pptx(provider_figs, [], year)
            label = f"Provider_Charts_{year}.pptx"
        else:
            _, _, provider_figs, _ = generate_monthly_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct, mc_sort_order)
            month_name = datetime.strptime(month.split('-')[1], '%m').strftime('%B')
            pptx_path = charts_to_pptx(provider_figs, [], f"{month_name} {year}")
            label = f"Provider_Charts_{month_name}_{year}.pptx"
        if pptx_path:
            return gr.File(value=pptx_path, label=label)
        else:
            print("Failed to generate provider PPTX")
            return None

    def download_employee_pptx(year, month, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if month == "Yearly":
            _, _, _, employee_figs = generate_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
            pptx_path = charts_to_pptx([], employee_figs, year)
            label = f"Employee_Charts_{year}.pptx"
        else:
            _, _, _, employee_figs = generate_monthly_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct, mc_sort_order)
            month_name = datetime.strptime(month.split('-')[1], '%m').strftime('%B')
            pptx_path = charts_to_pptx([], employee_figs, f"{month_name} {year}")
            label = f"Employee_Charts_{month_name}_{year}.pptx"
        if pptx_path:
            return gr.File(value=pptx_path, label=label)
        else:
            print("Failed to generate employee PPTX")
            return None

    scrape_btn.click(
        fn=scrape_and_store,
        inputs=[url_input, user_id_input, password_input, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            status_output, patient_state, claim_state, mc_state, year_dropdown, month_dropdown,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    year_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    month_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    show_mc_pct_checkbox.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    mc_sort_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    download_btn_prov.click(
        fn=download_provider_pptx,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

    download_btn_emp.click(
        fn=download_employee_pptx,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

demo.launch(share=True)

Overwriting claims_dashboard.py


In [1]:
!pip install pyinstaller

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   -------------------------------------- - 1.3/1.3 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 5.0 MB/s eta 0:00:00


In [3]:
!pip show webdriver-manager

Name: webdriver-manager
Version: 4.0.2
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packages
Requires: packaging, python-dotenv, requests
Required-by: 


In [2]:
!pyinstaller --onefile --add-data "C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packages\webdriver_manager;webdriver_manager" claims_dashboard.py

8517 INFO: PyInstaller: 6.12.0, contrib hooks: 2025.1
8517 INFO: Python: 3.12.7 (conda)
8559 INFO: Platform: Windows-10-10.0.19045-SP0
8559 INFO: Python environment: C:\Users\DELL-INTERN-HR\anaconda3
8564 INFO: wrote C:\Users\DELL-INTERN-HR\Documents\Dashboard\claims_dashboard.spec
8573 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\DELL-INTERN-HR\\anaconda3\\Scripts\\pyinstaller.exe',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\python312.zip',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\DLLs',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\Lib',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\DELL-INTERN-HR\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor',
 'C:\\Users\\DELL-INTERN-HR\\Documents\\Dashboard']
13946 I

In [3]:
!dir dist

 Volume in drive C is OS
 Volume Serial Number is 7E66-E747

 Directory of C:\Users\DELL-INTERN-HR\Documents\Dashboard\dist

02/26/2025  03:49 PM    <DIR>          .
02/26/2025  03:49 PM    <DIR>          ..
02/26/2025  03:51 PM       848,527,900 claims_dashboard.exe
               1 File(s)    848,527,900 bytes
               2 Dir(s)  38,036,090,880 bytes free


In [10]:
cd dist
claims_dashboard.exe  # Windows


SyntaxError: invalid syntax (4117908189.py, line 1)

In [ ]:
readme_content = """
Claims Analysis Dashboard
------------------------
1. Download claims_dashboard.exe (Windows) or claims_dashboard (macOS/Linux)
2. Double-click to run
3. Enter the website URL, user ID, and password in the app
4. Click "Submit" to scrape data and view charts
Requirements: Microsoft Edge browser installed, internet connection
Note: This app scrapes data from a specified website and displays charts locally.
"""
with open("README.txt", "w") as f:
    f.write(readme_content)

In [ ]:
import zipfile
with zipfile.ZipFile("ClaimsDashboard.zip", "w") as zf:
    zf.write("dist/claims_dashboard.exe")  # Windows
    zf.write("README.txt")